<a href="https://colab.research.google.com/github/Michael-Jimenez-C/CieCom2/blob/main/MapaCII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import folium
import numpy as np
from heapq import heappush, heappop
from typing import List



In [3]:
fc=lambda a: 2*np.arctan2(pow(a,1/2),pow(1-a,1/2))
fa=lambda lat1,long1,lat2,long2: pow(np.sin((lat2-lat1)/2),2)+np.cos(lat1)*np.cos(lat2)*pow(np.sin((long2-long1)/2),2)
fd=lambda A,B: 6378E1*fc(fa(A[0],A[1],B[0],B[1]))

In [4]:
class Nodo:
  def __init__(self,nombre,coordenadas):
    self.nombre=nombre
    self.coordenadas=coordenadas
    self.conexiones=[]
    self.status=0#0 no visitado, 1 visitado
    self.auxdist=np.infty
  
  def conectar(self,other):
    #fd=lambda A,B: pow(pow(A[0]-B[0],2)+pow(A[1]-B[1],2),1/2)
    if other not in self.conexiones:
      self.conexiones.append((other,fd(self.coordenadas,other.coordenadas)))
    if self not in other.conexiones:
      other.conexiones.append((self,fd(self.coordenadas,other.coordenadas)))
  

  def distancia(self,nombreNodo):
    for i in self.conexiones:
      if i[0].nombre==nombreNodo:
        return  [i[0],i[1]]
    distancia=np.infty
    d=[]
    for i in self.conexiones:
      if i[0].status==1:
        continue
      self.status=1
      k=i[0].distancia(nombreNodo)
      if k!=None:
        if i[1]+k[1]<distancia:
          distancia=i[1]+k[1]
          d=[i[0]]
          d.extend(([k[0]] if type(k[0])!=list else k[0]))
    self.status=0
    return [d,distancia]

  def __repr__(self):
    return self.nombre

In [5]:
A=Nodo('A',[0,1])
B=Nodo('B',[1,2])
C=Nodo('C',[2,1])
D=Nodo('D',[3,3])
E=Nodo('E',[3,2])
F=Nodo('F',[4,0])
G=Nodo('G',[4,2])
H=Nodo('H',[4,3])


In [6]:
A.conectar(B)
A.conectar(C)
B.conectar(C)
B.conectar(D)
B.conectar(E)
C.conectar(D)
C.conectar(F)
C.conectar(G)
D.conectar(H)
E.conectar(G)
G.conectar(H)

In [7]:
def matrrep(grafo):
  w={}
  for i in grafo:
    w[i.nombre]={j[0].nombre:j[1] for j in i.conexiones}
    for j in grafo:
      if j.nombre not in w[i.nombre]:
        w[i.nombre][j.nombre]=np.infty
  return w


In [34]:
'''
def dijkstra(matrrepg, origen, destino):
  def step(A,matrr):
    #copia del grafo
    matrrepg={i:matrr[i] for i in matrr}
    for i in matrrepg:
      matrrepg[i]={j:matrrepg[i][j] for j in matrrepg[i]}
    #Suma
    for i in A:
      for j in matrrepg[i]:
        matrrepg[i][j]+=A[i]
    for i in A:
      for j in matrrepg[i]:
        A[j]=np.minimum(A[j],matrrepg[i][j])

    if np.infty in [A[i] for i in A]:
      return step(A,matrr)
    return A

  for i in matrrepg:
    matrrepg[i].pop(origen)
  A=matrrepg.pop(origen)
  return step(A,matrrepg)
'''
def dijkstra(matrrepg, origen, destino):
  def step(A,matrr,con):
    for i in con:
      con[i].append(None)
    #copia del grafo
    matrrepg={i:matrr[i] for i in matrr}
    for i in matrrepg:
      matrrepg[i]={j:matrrepg[i][j] for j in matrrepg[i]}
    #Suma
    for i in A:
      for j in matrrepg[i]:
        matrrepg[i][j]+=A[i]
    for i in A:
      for j in matrrepg[i]:
        if matrrepg[i][j]<A[j]:
          A[j]=matrrepg[i][j]
          if j!=origen:
            con[j][-1]=i


    if np.infty in [A[i] for i in A]:
      return step(A,matrr,con)
    return A,con

  con={i:None for i in matrrepg}
  for i in matrrepg[origen]:
    con[i]=[origen if matrrepg[origen][i]!=np.infty else None]
  
  for i in matrrepg:
    matrrepg[i].pop(origen)
  A=matrrepg.pop(origen)
  distancias,conexiones=step(A,matrrepg,con)

  camino=[destino]
  N=destino
  while len(conexiones[origen])>0:
    N=conexiones[N][-1]
    camino.append(N)
    for i in conexiones:
      conexiones[i].pop()  

  return distancias[destino], list(reversed(camino))


In [35]:
distancia, ruta=dijkstra(matrrep([A,B,C,D,E,F,G,H]),'A','H')

(4.650281539872885, ['A', 'B', 'D', 'H'])

In [36]:
A.distancia("H")

[[B, D, H], 4.650281539872885]

#Mapa

In [38]:
puntos={"Bogota":([4.61,-74.082], "Soacha,La calera,Mosquera".split(",")),
        "Soacha":([4.583,-74.217], "Chipaque,Fusagasuga,La Mesa,Bogota".split(",")),
        "Madrid":([4.733,-74.267], "Facatativa,La Mesa,Mosquera,Funza".split(",")),
        "Facatativa":([4.817,-74.367], "Madrid,La Mesa,La Vega,Villeta".split(",")),
        "Fusagasuga":([4.333,-74.35], "Soacha".split(",")),
        "Funza":([4.717,-74.217], "Mosquera,Madrid,Cota".split(",")),
        "Chia":([4.85,-74.05], "Bogota,Cota,Gachancipa,Zipaquira".split(",")),
        "Zipaquira":([5.02836,-73.9992], "Gachancipa,Chia".split(",")),
        "Girardot":([4.3,-74.8], "Tocaima,Fusagasuga".split(",")),
        "Villeta":([5.017,-74.467], "La Vega,Facatativa".split(",")),
        "Macheta":([5.07983,-73.6072], "Gachancipa".split(",")),
        "La Vega":([4.99946,-74.3399], "Villeta,Bogota".split(",")),
        "Tocaima":([4.45903,-74.634], "Girardot,La Mesa".split(",")),
        "Mosquera":([4.70612,-74.2307], "Madrid,Funza,Bogota".split(",")),
        "La Mesa":([4.633,-74.467], "Soacha,Tocaima".split(",")),
        "Cota":([4.817,-74.1], "Chia,Funza".split(",")),
        "Gachancipa":([4.99228,-73.8712], "Macheta,Chia".split(",")),
        "La calera":([4.717,-73.967], "Bogota,Chipaque".split(",")),
        "Choachi":([4.52748,-73.9257], "Chipaque,La calera".split(",")),
        "Chipaque":([4.44294,-74.0445], "Choachi,Bogota".split(","))}

In [39]:
m=folium.Map(location=[4.61,-74.082], zoom_start=10, min_zoom=10)
for i in puntos:
  folium.Marker(
      puntos[i][0], popup=f"<i>{i}</i>"
  ).add_to(m)

for i in puntos:
  c=puntos[i]
  for j in c[1]:
    folium.PolyLine([c[0],puntos[j][0]], tooltip="").add_to(m)

In [40]:
nodos={i:Nodo(i,puntos[i][0]) for i in puntos}

In [41]:
for i in puntos:
  c=puntos[i]
  for j in c[1]:
    nodos[i].conectar(nodos[j])

In [ ]:
origen="Fusagasuga"
destino="Macheta"
ruta=nodos[origen].distancia(destino)
print(f"ruta{ruta[0]} Distancia{ruta[1]}")


ruta[0].insert(0,nodos[origen])

for j in range(len(ruta[0])-1):
  folium.PolyLine([ruta[0][j].coordenadas,ruta[0][j+1].coordenadas], color="red").add_to(m)
m

In [46]:
origen="Fusagasuga"
destino="Macheta"
ruta=dijkstra(matrrep([nodos[i] for i in nodos]),origen,destino)

print(f"ruta{ruta[1]} Distancia{ruta[0]}")
ruta=[nodos[i] for i in ruta[1]]

ruta.insert(0,nodos[origen])

for j in range(len(ruta)-1):
  folium.PolyLine([ruta[j].coordenadas,ruta[j+1].coordenadas], color="red").add_to(m)
m

ruta['Fusagasuga', 'Soacha', 'Bogota', 'Chia', 'Gachancipa', 'Macheta'] Distancia1.1696142697852183


In [ ]:
import heapq

def dijkstra(graph, start, end):
    # Inicialización de variables
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0
    visited = {}
    queue = [(0, start)]

    while queue:
        # Tomar el vértice con la distancia mínima
        current_distance, current_vertex = heapq.heappop(queue)

        # Si ya se visitó el vértice, continuar al siguiente
        if current_vertex in visited:
            continue

        visited[current_vertex] = True

        # Actualizar la distancia de los vértices adyacentes
        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(queue, (distance, neighbor))

    # Devolver la distancia entre los dos vértices
    return distances[end]

# Ejemplo de uso
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1},
    'E': {'B': 1, 'G': 6},
    'F': {'C': 5},
    'G': {'C': 7, 'E': 6, 'H': 2},
    'H': {'D': 3, 'G': 2}
}

start = 'F'
end = 'E'

distance = dijkstra(graph, start, end)

print(f"La distancia más corta entre {start} y {end} es: {distance}")


La distancia más corta entre F y E es: inf
